In [1]:
import fitz #PyMuPDF
import pandas as pd

from typing import List

In [2]:
def pdf_2_text(file_path: str) -> List[str]:
    """
    Extract text from a PDF file using PyMuPDF (fitz).

    Parameters:
     - file_path (str): Path to the PDF file.

    Returns:
     - List[str]: A list of strings, each representing the text extracted from a single page of the PDF.
    """
    doc = fitz.open(file_path)
    texts = []
    for page in doc:
        texts.append(page.get_text())
    doc.close()
    return texts

In [3]:
'''
def extract_minute_from_text(texts_pages: List[str], minute_version: int, perfect_segregation: bool, initial_page: int) -> str:
    """
    Extract the Copom minute text from the list of text of each page of the .pdf file.
    
    Parameters:
     - texts_pages (List[str]): List of strings, each representing the text of a page.
     - minute_version (int): The version of the Copom minute.
     - perfect_segregation (bool): If True, consider that the minute starts at a specific page, with no noise.
     - initial_page (int): The page number where the minute starts.

    Returns:
     - str: The extracted Copom minute text.
    """
    if perfect_segregation == 1:
        texts_pages = texts_pages[initial_page - 1:]

        if minute_version == 4 or minute_version == 5:
            try:
                start_index = texts_pages[0].find("A) ")
            except:
                print("ERRO")
        else:
            start_index = 0
        texts_pages = [item[start_index:] for item in texts_pages]
        
        #problem here? code mantain the footnotes...

        copom_minute_text = "\n".join(texts_pages)
        return copom_minute_text
    
    else:
        texts_pages = texts_pages[initial_page - 1:]

        if minute_version == 1:
            try:
                start_index = texts_pages[0].find("The Board analyzed ")
                texts_pages[0] = texts_pages[0][start_index:]
            except:
                print("ERRO")

        elif minute_version == 2:
            try:
                start_index = texts_pages[0].find("The Board analyzed ")
                texts_pages[0] = texts_pages[0][start_index:]
            except:
                print("ERRO")

            try:
                texts_pages_temp = texts_pages.copy()
                for i in len(texts_pages_temp):
                    end_index = texts_pages_temp[i].find("Acronyms")
                    if end_index == -1:
                        pass
                    else:
                        texts_pages[i] = texts_pages[i][:end_index]
                        texts_pages = texts_pages[:i]
                        continue
            except:
                print("ERRO")
            
            
        texts_pages = [item[:] for item in texts_pages]
        copom_minute_text = "\n".join(texts_pages)
        return copom_minute_text
    
'''

'\ndef extract_minute_from_text(texts_pages: List[str], minute_version: int, perfect_segregation: bool, initial_page: int) -> str:\n    """\n    Extract the Copom minute text from the list of text of each page of the .pdf file.\n\n    Parameters:\n     - texts_pages (List[str]): List of strings, each representing the text of a page.\n     - minute_version (int): The version of the Copom minute.\n     - perfect_segregation (bool): If True, consider that the minute starts at a specific page, with no noise.\n     - initial_page (int): The page number where the minute starts.\n\n    Returns:\n     - str: The extracted Copom minute text.\n    """\n    if perfect_segregation == 1:\n        texts_pages = texts_pages[initial_page - 1:]\n\n        if minute_version == 4 or minute_version == 5:\n            try:\n                start_index = texts_pages[0].find("A) ")\n            except:\n                print("ERRO")\n        else:\n            start_index = 0\n        texts_pages = [item[sta

In [228]:
def extract_minute_from_text_2(texts_pages: List[str], minute_version: int, perfect_segregation: bool, initial_page: int) -> str:
    """
    Extract the Copom minute text from the list of text of each page of the .pdf file.
    
    Parameters:
     - texts_pages (List[str]): List of strings, each representing the text of a page.
     - minute_version (int): The version of the Copom minute.
     - perfect_segregation (bool): If True, consider that the minute starts at a specific page, with no noise.
     - initial_page (int): The page number where the minute starts.

    Returns:
     - str: The extracted Copom minute text.
    """

    texts_pages = texts_pages[initial_page - 1:]
    texts_pages_adjusted = []

    if perfect_segregation == 1:
        
        if minute_version == 5: #Done
            for page in texts_pages:
                page = page.replace('\xa0', ' ') # or convert to ascii?
                start_index = page.find("bcb.gov.br")
                if start_index == -1:
                    print("ERRO")
                texts_pages_adjusted.append(page[start_index+12:])


        elif minute_version == 4: #Done
            for page in texts_pages:
                page = page.replace('\xa0', ' ') # or convert to ascii?
                start_index = page.find("Copom Meeting")
                plus_index = 18
                flag_continue = True
                while flag_continue:
                    if (page[start_index+plus_index:start_index+plus_index+1] == "\n") and (page[start_index+plus_index:start_index+plus_index+2] != "\n "):
                        flag_continue = False
                    else:
                        plus_index += 1

                if start_index == -1:
                    print("ERRO")
                    
                texts_pages_adjusted.append(page[start_index+plus_index:])

    else:
        if minute_version == 3:
            page_1_text_upper = texts_pages[0].upper()
            start_index_page_1 = page_1_text_upper.find("THE MEMBERS OF THE COPOM ANALYZED ")
            if start_index_page_1 == -1:
                start_index_page_1 = page_1_text_upper.find("THE MEMBERS OF THE MONETARY POLICY ")
            texts_pages_adjusted.append(texts_pages[0][start_index_page_1:])
            
            for page in texts_pages[1:]:
                start_index = 0
                flag_continue = True
                while flag_continue:
                    if (page[start_index:start_index+1] == "\n") and (page[start_index:start_index+2] != "\n "):
                        flag_continue = False
                    else:
                        start_index += 1

                if start_index == -1:
                    print("ERRO")
                
                texts_pages_adjusted.append(page[start_index+1:])
                '''
                page = page.replace('\xa0', ' ') # or convert to ascii?
                if start_index == -1:
                    print("ERRO")
                print(page[:300])    
                texts_pages_adjusted.append(page[start_index+12:])
                '''


    '''
        else:   
            texts_pages_adjusted = [item for item in texts_pages]

    else:
        pass

    
    else:
        if minute_version == 1:
            try:
                start_index = texts_pages[0].find("The Board analyzed ")
                texts_pages[0] = texts_pages[0][start_index:]
            except:
                print("ERRO")

        elif minute_version == 2:
            try:
                start_index = texts_pages[0].find("The Board analyzed ")
                texts_pages[0] = texts_pages[0][start_index:]
                print("ERRO")
            except:
                pass

            try:
                texts_pages_temp = texts_pages.copy()
                for i in len(texts_pages_temp):
                    end_index = texts_pages_temp[i].find("Acronyms")
                    if end_index == -1:
                        pass
                    else:
                        texts_pages[i] = texts_pages[i][:end_index]
                        texts_pages = texts_pages[:i]
                        continue
            except:
                print("ERRO")
    '''

    for page in texts_pages_adjusted:
        print(page[:100])
        print("############")

    copom_minute_text = "\n".join(texts_pages_adjusted)
    return copom_minute_text

In [229]:
raw_folder_path = "./data/copom_minutes_raw"

In [237]:
df_minutes_format = pd.read_excel("minutes_format.xlsx")
df_minutes_format = df_minutes_format[df_minutes_format["Version"] == 3]
df_minutes_format.tail(50)

,Titulo,Version,PerfSeg,InitialPage
107,150th Copom Minutes,3,0,1
108,151st Copom Minutes,3,0,1
109,152nd Copom Minutes,3,0,1
110,153rd Copom Minutes,3,0,1
111,154th Copom Minutes,3,0,1
112,155th Copom Minutes,3,0,1
113,156th Copom Minutes,3,0,1
114,157th Copom Minutes,3,0,1
115,158th Copom Minutes,3,0,1
116,159th Copom Minutes,3,0,1


In [239]:
output_folder_path = "./data/copom_minutes_texts"


for index, row in df_minutes_format.iterrows():
    if index == 156:
        minute_version = row.Version
        perfect_segregation = row.PerfSeg
        initial_page = row.InitialPage

        texts_pages = pdf_2_text(f"{raw_folder_path}/{row.Titulo}.pdf")

        minute_text = extract_minute_from_text_2(texts_pages, minute_version, perfect_segregation, initial_page)

        #print(copom_minute_text)

    '''
    try:
        with open(f"{output_folder_path}/{row.Titulo}.txt", 'w', encoding='utf-8') as f:
            f.write(minute_text)
        print(f"[INFO] Sucess saving file: {row.Titulo}")

    except IOError as e:
        print(f"[INFO] Error saving {row.Titulo}: {e}")
    '''

The members of the Copom analyzed the recent performance and the prospects for the Brazilian economy
############
2
(0.85 p.p. above the one registered in May 2015). The double weight core inflation changed from 0.
############
3
 
8. According to the Continuous National Household Sample Survey (PNADC), the unemployment rate i
############
4
15. For the set of regulated prices inflation, the projected change is 6.8% in 2016, unchanged rel
############
5
 
22. The Committee considers that the external environment remains especially complex. However, d
############
6
expansion, consumer credit has moderated, so that, over the last quarters, there were, on the one 
############
